# Deep Learning Optimization

#### Goal of This File:

##### 1. Import Libraries and Helper Functions ==> Data Sourcing and Munging ==> Utility Functions ==> Data Preparation and Pre-Processing

##### 2. Confirm Findings From ML File ==> Sklearn Embeddings vs Gensim Embeddings ==> Sentiment Polarity Final Score ==> 3 labels vs 5 labels

##### 3. XGBoost

##### 4. Simple RNN

##### 5. LSTM ==> Bidirectional LSTM

##### 6. Sequence-to-Vector Transformers

##### 7. Conclusion

## 1. Import Libraries, Helper Functions and Load Constants

In [1]:
#this value checks if vast is being used
VAST = True

In [2]:
if VAST:
    !pip install pandas numpy nltk scikit-learn wordcloud seaborn gensim tensorflow matplotlib > /dev/null


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
import requests
import os
import tensorflow as tf
TF_ENABLE_ONEDNN_OPTS=0
from nlp_ai_utils import *
from chunks_urls import CHUNKS_URLS
URLS = CHUNKS_URLS
LIMIT = 60
BASE_FILE_NAME = "chunk_"
ACTUAL_DATA_SHAPE = 6990280
CHUNKS_COUNT = 60

2023-04-27 02:54:59.862762: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-27 02:54:59.893364: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### 1.1 Data Sourcing and Munging

#### 1.1.1 Utility Functions

In [4]:
def get_chunks(urls,limit=False,verbose = 1,base_name = ""):
    #downloads all data from their urls
    for i,url in enumerate(urls):
        if limit:
            if i == limit - 1:
                return
        file_name = base_name + str(i + 1)
        #checks if file already exists
        if os.path.exists(file_name):
            print(f"{file_name} already exists.")
            continue
        r = requests.get(url)
        if i % verbose == 0:
            print(f"Downloading {file_name}...")
        with open(file_name, 'wb') as fd:
            for chunk in r.iter_content():
                #save file in the current directory of the notebook
                fd.write(chunk)

In [5]:
def get_all_file_names(base_name,limit_num):
    return [base_name + str(num) for num in range(1,limit_num + 1)]

In [6]:
def read_chunks(files):
    #reads chunks csvs and converts them to a dataframe format
    final_df = []
    for file in files:
        df = pd.read_csv(file)
        final_df.append(df)
        #function returns a list of dfs, or one df if only one input is given
    return final_df

In [7]:
def group_up_chunks(dfs):
    #adds up all dataframes together
    return pd.concat(dfs)

In [8]:
def check_no_missing_data(shape_loaded,shape_actual):
    actual_shape_loaded = (shape_loaded / shape_actual) * 100
    return actual_shape_loaded

#### 1.1.2 Loading The Data

In [9]:
get_chunks(URLS,0,1,BASE_FILE_NAME)

chunk_1 already exists.
chunk_2 already exists.
chunk_3 already exists.
chunk_4 already exists.
chunk_5 already exists.
chunk_6 already exists.
chunk_7 already exists.
chunk_8 already exists.
chunk_9 already exists.
chunk_10 already exists.
chunk_11 already exists.
chunk_12 already exists.
chunk_13 already exists.
chunk_14 already exists.
chunk_15 already exists.
chunk_16 already exists.
chunk_17 already exists.
chunk_18 already exists.
chunk_19 already exists.
chunk_20 already exists.
chunk_21 already exists.
chunk_22 already exists.
chunk_23 already exists.
chunk_24 already exists.
chunk_25 already exists.
chunk_26 already exists.
chunk_27 already exists.
chunk_28 already exists.
chunk_29 already exists.
chunk_30 already exists.
chunk_31 already exists.
chunk_32 already exists.
chunk_33 already exists.
chunk_34 already exists.
chunk_35 already exists.
chunk_36 already exists.
chunk_37 already exists.
chunk_38 already exists.
chunk_39 already exists.
chunk_40 already exists.
chunk_41 

#### 1.1.3 Merging all Individual Files

In [10]:
#get all names of downloaded files
all_file_names = get_all_file_names(BASE_FILE_NAME,CHUNKS_COUNT)

In [11]:
#read all chunks into a list
list_dfs = read_chunks(all_file_names)

In [12]:
#concatenate all chunks into a singular df
df = group_up_chunks(list_dfs)

In [13]:
#check how much of the data was actually downloaded
percent_loaded = check_no_missing_data(df.shape[0],ACTUAL_DATA_SHAPE)
percent_loaded

51.66681449097891

### 1.2 Data Preparation and Pre-processing